# Creating FABnet IPv4 Network: Fully Automatic Configuration

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). You can think of this service as a private internet that connects experiments across the testbed using FABRIC's high-performance network links. 

This notebook describes how to use the FABnetv4 service which is FABRIC's private IPv4 internet using FABlib's the fully automatic configuration.   


## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet.  

Two nodes are created and each is automatically connected to a FABnetv4 network.  Fully automatic configuration does not require the user to manually add or configure interfaces. Instead, the `node.add_fabnet()` function is called directly on the node.  The call adds an interface and connects it to a FABnet network on the local site.  If multiple nodes reside on the same site, they will be connected to the same FABnet network.   The interface's mode is set to `auto` enabling an automatic configuration of the IP addresses during the post boot phase.

This is the simplest way to connect your nodes to FABnet and is recommended for anyone who does not need custom control over IPs and routes.


In [ ]:
slice_name = 'MySlice'
[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
node1.add_fabnet()

# Node2
node2 = slice.add_node(name=node2_name, site=site2)
node2.add_fabnet()

#Submit Slice Request
slice.submit();

## Run the Experiment

With automatic configuration the slice is ready for experimentation after it becomes active.  Note that automatic configuration works well when saving slices to a file and reinstantiating the slice.  Configuration tasks can be stored in the saved slice, reducing the complexity of notebooks and other runtime steps.

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=f'FABNET_IPv4_{node2.get_site()}').get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
    



## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()